<a href="https://colab.research.google.com/github/satvik-venkatesh/word-eq/blob/main/Word-EQ-GloVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil

In [ ]:
import pandas as pd

In [1]:
"""
Download the Glove from drive.
"""
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1A-KM4ugWdi3rJ8lVKYZXdrAcfZQlkGj-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1A-KM4ugWdi3rJ8lVKYZXdrAcfZQlkGj-" -O glove.6B.zip && rm -rf /tmp/cookies.txt


--2021-12-10 16:39:43--  https://docs.google.com/uc?export=download&confirm=bmAZ&id=1A-KM4ugWdi3rJ8lVKYZXdrAcfZQlkGj-
Resolving docs.google.com (docs.google.com)... 142.250.141.139, 142.250.141.113, 142.250.141.101, ...
Connecting to docs.google.com (docs.google.com)|142.250.141.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-2o-docs.googleusercontent.com/docs/securesc/bfhakek5oodf9897q1k2rqv1bovvq364/i3eqtmnlop7c0131015fotisuvfprsr4/1639154325000/04739181468756608208/06236756351345617880Z/1A-KM4ugWdi3rJ8lVKYZXdrAcfZQlkGj-?e=download [following]
--2021-12-10 16:39:43--  https://doc-14-2o-docs.googleusercontent.com/docs/securesc/bfhakek5oodf9897q1k2rqv1bovvq364/i3eqtmnlop7c0131015fotisuvfprsr4/1639154325000/04739181468756608208/06236756351345617880Z/1A-KM4ugWdi3rJ8lVKYZXdrAcfZQlkGj-?e=download
Resolving doc-14-2o-docs.googleusercontent.com (doc-14-2o-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Conne

In [2]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

In [3]:
#!wget https://nlp.stanford.edu/data/glove.840B.300d.zip
"""
Download the Glove 840B from drive.
"""
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=18SddG0mv6bfRItzTUzRrlsuqLmtzS55r' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=18SddG0mv6bfRItzTUzRrlsuqLmtzS55r" -O glove.840B.300d.zip && rm -rf /tmp/cookies.txt

--2021-12-10 16:40:45--  https://docs.google.com/uc?export=download&confirm=hLgf&id=18SddG0mv6bfRItzTUzRrlsuqLmtzS55r
Resolving docs.google.com (docs.google.com)... 142.250.141.102, 142.250.141.101, 142.250.141.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.141.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-7c-docs.googleusercontent.com/docs/securesc/gt3qjjlr6va9uj8v7nqs7jmc6tqt0h3d/vecf49q4u3bg8olbpu7ml5oi6r9e3sh1/1639154400000/04739181468756608208/04546961575664016603Z/18SddG0mv6bfRItzTUzRrlsuqLmtzS55r?e=download [following]
--2021-12-10 16:40:45--  https://doc-00-7c-docs.googleusercontent.com/docs/securesc/gt3qjjlr6va9uj8v7nqs7jmc6tqt0h3d/vecf49q4u3bg8olbpu7ml5oi6r9e3sh1/1639154400000/04739181468756608208/04546961575664016603Z/18SddG0mv6bfRItzTUzRrlsuqLmtzS55r?e=download
Resolving doc-00-7c-docs.googleusercontent.com (doc-00-7c-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Conne

In [4]:
!unzip -q glove.840B.300d.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !cp "/content/drive/MyDrive/audio-and-word-embeddings/eq_contributions.csv" .

In [7]:
import os
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
import csv
from collections import Counter
import random
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import matplotlib.pyplot as plt

In [10]:
def read_annotation(filename):
    events = []
    with open(filename, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            events.append(row)
    return events

In [5]:
"""
Download the SocialFX dataset for EQ settings and descriptors. 
"""
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1BrI0XPhzwDt7-_PZwtpsavtoPvRvjyKF' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1BrI0XPhzwDt7-_PZwtpsavtoPvRvjyKF" -O eq_contributions.csv && rm -rf /tmp/cookies.txt

--2021-12-10 16:43:50--  https://docs.google.com/uc?export=download&confirm=&id=1BrI0XPhzwDt7-_PZwtpsavtoPvRvjyKF
Resolving docs.google.com (docs.google.com)... 142.250.141.113, 142.250.141.139, 142.250.141.138, ...
Connecting to docs.google.com (docs.google.com)|142.250.141.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-bs-docs.googleusercontent.com/docs/securesc/3bkssvtrg742kl603ksgq529a6136kof/qdspvvs7dvi5p8eons4bdk30r6l18amm/1639154625000/04739181468756608208/13263907668047105798Z/1BrI0XPhzwDt7-_PZwtpsavtoPvRvjyKF?e=download [following]
--2021-12-10 16:43:52--  https://doc-00-bs-docs.googleusercontent.com/docs/securesc/3bkssvtrg742kl603ksgq529a6136kof/qdspvvs7dvi5p8eons4bdk30r6l18amm/1639154625000/04739181468756608208/13263907668047105798Z/1BrI0XPhzwDt7-_PZwtpsavtoPvRvjyKF?e=download
Resolving doc-00-bs-docs.googleusercontent.com (doc-00-bs-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connectin

In [11]:
events = read_annotation("/content/eq_contributions.csv")
len(events)

1596

In [12]:
words_list = [e[0].lower() for e in events[1:] if e[1] == "English"]

In [13]:
len(words_list)

918

In [14]:
hq_words = ["bright","brittle","clear","crisp","harsh","hollow","sharp","shrill","tinny","woody","big","boom","boxy","dark","dull","fat","full","muddy","muffled","punch","smooth","sweet","warm","flat",
            "crunchy","deep","soothing","clean","airy","cold","metallic","booming"]

In [15]:
len(hq_words)

32

In [16]:
words_list[0:10]

['hot',
 'wet',
 'slow',
 'saucy',
 'heavy',
 'shrill',
 'warm',
 'chill',
 'sharp',
 'corn']

In [17]:
high_rated_words = list(set([e[0].lower() for e in events[1:] if e[1] == "English" and float(e[3]) > 0.7 and e[0].lower() not in hq_words]))
high_rated_words.sort()

In [18]:
high_rated_words

['aggressive',
 'baseball',
 'beautiful',
 'biting',
 'bold',
 'brash',
 'brass',
 'brassy',
 'breezy',
 'calm',
 'caring',
 'cheerful',
 'clarity',
 'cooing',
 'cool',
 'cute',
 'disgusting',
 'down',
 'edge',
 'edgy',
 'enchanting',
 'energetic',
 'energizing',
 'excited',
 'exciting',
 'fancy',
 'fluffy',
 'forceful',
 'frigid',
 'funky',
 'genius',
 'gentle',
 'good',
 'gruff',
 'happy',
 'hard',
 'heart-warming',
 'heat',
 'heavy',
 'hot',
 'huge',
 'icy',
 'jagged',
 'large',
 'light',
 'loud',
 'love',
 'low',
 'mellow',
 'mournful',
 'noisy',
 'passionate',
 'peace',
 'peaceful',
 'pleasing',
 'pleasurable',
 'plodding',
 'poor',
 'punchy',
 'quiet',
 'radiant',
 'relaxing',
 'reserved',
 'rich',
 'romantic',
 'rousing',
 'rumble',
 'serene',
 'sloppy',
 'slow',
 'soft',
 'solemn',
 'splash',
 'squeaking',
 'strong',
 'taco',
 'techno',
 'tense',
 'throbbing',
 'thumpy',
 'thunderous',
 'twangy',
 'velvety',
 'whispered',
 'whispering',
 'wistful']

In [19]:
len(high_rated_words)

86

In [20]:
# 5, 19, 71
# Using seed 137 for shuffling into folds
random.seed(137)
random.shuffle(hq_words)
random.shuffle(high_rated_words)
#test_word_set = hq_words[0:10] + high_rated_words[0:30]

In [21]:
test_word_set_1 = hq_words[0:9] + high_rated_words[0:22]
test_word_set_2 = hq_words[9:18] + high_rated_words[22:44]
test_word_set_3 = hq_words[18:27] + high_rated_words[44:66]

In [22]:
test_word_set_4 = hq_words[27:] + hq_words[0:4] + high_rated_words[66:] + high_rated_words[0:2]

In [23]:
test_word_set_4

['sharp',
 'big',
 'dark',
 'hollow',
 'harsh',
 'smooth',
 'muffled',
 'crisp',
 'punch',
 'mournful',
 'clarity',
 'genius',
 'bold',
 'twangy',
 'soft',
 'splash',
 'slow',
 'wistful',
 'brash',
 'fancy',
 'cute',
 'rousing',
 'loud',
 'breezy',
 'large',
 'passionate',
 'baseball',
 'huge',
 'icy',
 'brassy',
 'caring']

In [24]:
cc = dict(Counter(words_list))

In [25]:
words_set = list(cc.keys())
words_set.sort()

In [26]:
len(words_set)

388

In [27]:
cc

{'aggravating': 2,
 'aggressive': 4,
 'agitated': 1,
 'aidan': 1,
 'airy': 3,
 'alluring': 1,
 'anger': 1,
 'angry': 2,
 'annoying': 1,
 'anticipation': 1,
 'apple': 1,
 'archaic': 1,
 'awesome': 2,
 'awkward': 1,
 'bag': 1,
 'balanced': 1,
 'bang': 1,
 'baseball': 1,
 'bassy': 1,
 'bear': 1,
 'beautiful': 10,
 'big': 1,
 'biting': 1,
 'blaring': 1,
 'blast': 1,
 'boisterous': 1,
 'bold': 2,
 'bonito': 1,
 'boom': 1,
 'booming': 1,
 'bored': 1,
 'bouncy': 1,
 'boxy': 1,
 'brash': 1,
 'brass': 1,
 'brassy': 2,
 'brave': 1,
 'breezy': 1,
 'bright': 19,
 'brillante': 1,
 'brittle': 1,
 'buzz': 2,
 'calm': 13,
 'car': 1,
 'carefree': 1,
 'caring': 1,
 'cat': 2,
 'caustic': 1,
 'cerebral': 1,
 'charming': 1,
 'cheerful': 3,
 'cheery': 1,
 'children': 1,
 'chill': 1,
 'chilly': 1,
 'choppy': 1,
 'chunky': 2,
 'clarity': 1,
 'clash': 1,
 'classic': 2,
 'classy': 1,
 'clean': 4,
 'clear': 12,
 'coarse': 1,
 'cold': 34,
 'computer': 1,
 'confident': 1,
 'congested': 1,
 'cooing': 1,
 'cool': 14

# Seeds are changed below

In [28]:
train_word_set = [w for w in words_set if w not in test_word_set_1]

In [29]:
train_events = [e for e in events[1:] if e[1] == "English" and e[0].lower() in train_word_set]
#val_events = [e for e in events[1:] if e[1] == "English" and e[0] in val_word_set]
test_events = [e for e in events[1:] if e[1] == "English" and e[0].lower() in test_word_set_1 and float(e[3]) > 0.7]

In [32]:
words_set = list(cc.keys())
words_set.sort()

In [33]:
train_samples = [e[0].lower() for e in train_events]
train_labels = [e[4:] for e in train_events]

# val_samples = [e[0] for e in val_events]
# val_labels = [e[4:] for e in val_events]

test_samples = [e[0].lower() for e in test_events]
test_labels = [e[4:] for e in test_events]

In [34]:
print(len(train_events))
# print(len(val_events))
print(len(test_events))

792
38


In [35]:
"warm" in train_word_set

True

In [37]:
cc['bright']

19

In [38]:
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), "/content/glove.840B.300d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  if __name__ == '__main__':


KeyboardInterrupt: ignored

In [ ]:
vec2 = TextVectorization(max_tokens=400010, output_sequence_length=1)
eq_ds = tf.data.Dataset.from_tensor_slices(words_set).batch(128)
vec2.adapt(eq_ds)

In [ ]:
import spacy

In [ ]:
# download and import the large english model.
!python -m spacy download en_core_web_lg
import en_core_web_lg

In [ ]:
nlp = en_core_web_lg.load()
vec2 = TextVectorization(output_sequence_length=1)
eq_ds = tf.data.Dataset.from_tensor_slices(words_set).batch(128)
vec2.adapt(eq_ds)

In [ ]:
import pandas as pd

In [ ]:
Vectorizer = TextVectorization()

#load the data 
text = pd.read_csv('https://github.com/Violet-Spiral/assessing-childrens-writing/raw/main/data/samples_no_title.csv').dropna()

#fit the vectorizer on the text and extract the corpus vocabulary
Vectorizer.adapt(text.Text.to_numpy())
vocab = Vectorizer.get_vocabulary()

#generate the embedding matrix
num_tokens = len(vocab)
embedding_dim = len(nlp('The').vector)
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for i, word in enumerate(vocab):
    embedding_matrix[i] = nlp(str(word)).vector

In [ ]:
vec2 = TextVectorization(max_tokens=400010, output_sequence_length=1)
eq_ds = tf.data.Dataset.from_tensor_slices(list(embeddings_index.keys())).batch(128)
vec2.adapt(eq_ds)

In [ ]:
voc2 = vec2.get_vocabulary()
word_index_2 = dict(zip(voc2, range(len(voc2))))

In [ ]:
len(voc2)

In [ ]:
word_index_2

In [ ]:
#generate the embedding matrix for spacy
num_tokens = len(voc2)
embedding_dim = len(nlp('The').vector)
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for i, word in enumerate(word_index_2):
    embedding_matrix[i] = nlp(str(word)).vector

In [ ]:
em,hit = read_annotation_2("/content/dict2vec300/dict2vec-300d.vec", "warm")

In [ ]:
em

In [ ]:
def read_annotation_2(filename, word):
    events = []
    with open(filename, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in spamreader:
            if row[0] == word:
              return row,True
        return -1,False

In [ ]:
# Embedding matrix for dict2vec
#generate the embedding matrix

voc2 = vec2.get_vocabulary()
word_index_2 = dict(zip(voc2, range(len(voc2))))

num_tokens = len(voc2)
embedding_dim = 300
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for i, word in enumerate(word_index_2):
    if word == '' or word == '[UNK]':
      embedding_matrix[i] = np.zeros(embedding_dim,)
      print("Missed: " + word)
      continue
    em,hit = read_annotation_2("/content/dict2vec-300d.vec", word)
    if not hit:
      embedding_matrix[i] = np.zeros(embedding_dim,)
      print("Missed: " + word)

    else:
      em_2 = np.array(em[1:-1]).astype(float)
      embedding_matrix[i] = em_2
      print("Added: " + word)

In [ ]:
max(nlp('keyboard').vector)

In [ ]:
type(word_index_2)

In [ ]:
# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index_2.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        print(word)
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
vec2.get_vocabulary()[:10]

In [ ]:
output = vec2([["loud"]])
output.numpy()[0, :6]

In [ ]:
voc2 = vec2.get_vocabulary()
word_index_2 = dict(zip(voc2, range(len(voc2))))

In [ ]:
voc2

In [ ]:
len(voc2)

In [ ]:
test = ["warm", "cold", "soft"]
[word_index_2[w] for w in test]

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
plt.plot(embeddings_index['hot'])
plt.plot(embeddings_index['cold'])
plt.plot(embeddings_index['ocean'])

In [ ]:
cosine(embeddings_index['fire'], embeddings_index['burn'])

In [ ]:
embedding_matrix = np.random.rand(num_tokens, embedding_dim)

In [ ]:
num_tokens = len(voc2) + 2
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix for GloVe
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index_2.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        print(word)
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
# x_train = vec2(np.array([[s] for s in train_samples])).numpy()
# x_val = vec2(np.array([[s] for s in val_samples])).numpy()

# y_train = np.array(train_labels, dtype='float')
# y_val = np.array(val_labels, dtype='float')

# Data is normalised below

In [ ]:
x_train = vec2(np.array([[s] for s in train_samples])).numpy()
#x_val = vec2(np.array([[s] for s in val_samples])).numpy()
x_test = vec2(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels, dtype='float')[:, :]
#y_val = np.array(val_labels, dtype='float')[:, :]
y_test = np.array(test_labels, dtype='float')[:, :]

In [ ]:
y_train.shape

In [ ]:
mmin = np.ones((40,)) * -4
mmin

In [ ]:
mmax = np.ones((40,)) * 4
mmax

In [ ]:
# y_combined = np.concatenate((y_train, y_val, y_test), axis = 0)
# mmax = y_combined.max(axis=0)
# mmin = y_combined.min(axis=0)

In [ ]:
# mmax = np.ones()

In [ ]:
y_train = (y_train - mmin) / (mmax - mmin)
#y_val = (y_val - mmin) / (mmax - mmin)
y_test = (y_test - mmin) / (mmax - mmin)

In [ ]:
print(np.max(y_train))
#print(np.max(y_val))
print(np.max(y_test))

In [ ]:
num_tokens = 392
embedding_dim = 300

In [ ]:
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True)

In [ ]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    # embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    # trainable=False,
)

In [ ]:
num_tokens, embedding_dim

In [ ]:
def mean_abs_error_0(y_true, y_pred):
  diff = tf.abs(y_pred[:, 0] - y_true[:, 0])
  return tf.reduce_mean(diff)

def mean_abs_error_1(y_true, y_pred):
  diff = tf.abs(y_pred[:, 1] - y_true[:, 1])
  return tf.reduce_mean(diff)

def mean_abs_error_2(y_true, y_pred):
  diff = tf.abs(y_pred[:, 2] - y_true[:, 2])
  return tf.reduce_mean(diff)

def mean_abs_error_3(y_true, y_pred):
  diff = tf.abs(y_pred[:, 3] - y_true[:, 3])
  return tf.reduce_mean(diff)

def mean_abs_error_4(y_true, y_pred):
  diff = tf.abs(y_pred[:, 4] - y_true[:, 4])
  return tf.reduce_mean(diff)

def mean_abs_error_5(y_true, y_pred):
  diff = tf.abs(y_pred[:, 5] - y_true[:, 5])
  return tf.reduce_mean(diff)

def mean_abs_error_6(y_true, y_pred):
  diff = tf.abs(y_pred[:, 6] - y_true[:, 6])
  return tf.reduce_mean(diff)

def mean_abs_error_7(y_true, y_pred):
  diff = tf.abs(y_pred[:, 7] - y_true[:, 7])
  return tf.reduce_mean(diff)

def mean_abs_error_8(y_true, y_pred):
  diff = tf.abs(y_pred[:, 8] - y_true[:, 8])
  return tf.reduce_mean(diff)

def mean_abs_error_9(y_true, y_pred):
  diff = tf.abs(y_pred[:, 9] - y_true[:, 9])
  return tf.reduce_mean(diff)

def mean_abs_error_10(y_true, y_pred):
  diff = tf.abs(y_pred[:, 10] - y_true[:, 10])
  return tf.reduce_mean(diff)

def mean_abs_error_11(y_true, y_pred):
  diff = tf.abs(y_pred[:, 11] - y_true[:, 11])
  return tf.reduce_mean(diff)

def mean_abs_error_12(y_true, y_pred):
  diff = tf.abs(y_pred[:, 12] - y_true[:, 12])
  return tf.reduce_mean(diff)

def mean_abs_error_13(y_true, y_pred):
  diff = tf.abs(y_pred[:, 13] - y_true[:, 13])
  return tf.reduce_mean(diff)

def mean_abs_error_14(y_true, y_pred):
  diff = tf.abs(y_pred[:, 14] - y_true[:, 14])
  return tf.reduce_mean(diff)

def mean_abs_error_15(y_true, y_pred):
  diff = tf.abs(y_pred[:, 15] - y_true[:, 15])
  return tf.reduce_mean(diff)

def mean_abs_error_16(y_true, y_pred):
  diff = tf.abs(y_pred[:, 16] - y_true[:, 16])
  return tf.reduce_mean(diff)

def mean_abs_error_17(y_true, y_pred):
  diff = tf.abs(y_pred[:, 17] - y_true[:, 17])
  return tf.reduce_mean(diff)

def mean_abs_error_18(y_true, y_pred):
  diff = tf.abs(y_pred[:, 18] - y_true[:, 18])
  return tf.reduce_mean(diff)

def mean_abs_error_19(y_true, y_pred):
  diff = tf.abs(y_pred[:, 19] - y_true[:, 19])
  return tf.reduce_mean(diff)

def mean_abs_error_20(y_true, y_pred):
  diff = tf.abs(y_pred[:, 20] - y_true[:, 20])
  return tf.reduce_mean(diff)

def mean_abs_error_21(y_true, y_pred):
  diff = tf.abs(y_pred[:, 21] - y_true[:, 21])
  return tf.reduce_mean(diff)

def mean_abs_error_22(y_true, y_pred):
  diff = tf.abs(y_pred[:, 22] - y_true[:, 22])
  return tf.reduce_mean(diff)

def mean_abs_error_23(y_true, y_pred):
  diff = tf.abs(y_pred[:, 23] - y_true[:, 23])
  return tf.reduce_mean(diff)

def mean_abs_error_24(y_true, y_pred):
  diff = tf.abs(y_pred[:, 24] - y_true[:, 24])
  return tf.reduce_mean(diff)

def mean_abs_error_25(y_true, y_pred):
  diff = tf.abs(y_pred[:, 25] - y_true[:, 25])
  return tf.reduce_mean(diff)

def mean_abs_error_26(y_true, y_pred):
  diff = tf.abs(y_pred[:, 26] - y_true[:, 26])
  return tf.reduce_mean(diff)

def mean_abs_error_27(y_true, y_pred):
  diff = tf.abs(y_pred[:, 27] - y_true[:, 27])
  return tf.reduce_mean(diff)

def mean_abs_error_28(y_true, y_pred):
  diff = tf.abs(y_pred[:, 28] - y_true[:, 28])
  return tf.reduce_mean(diff)

def mean_abs_error_29(y_true, y_pred):
  diff = tf.abs(y_pred[:, 29] - y_true[:, 29])
  return tf.reduce_mean(diff)

def mean_abs_error_30(y_true, y_pred):
  diff = tf.abs(y_pred[:, 30] - y_true[:, 30])
  return tf.reduce_mean(diff)

def mean_abs_error_31(y_true, y_pred):
  diff = tf.abs(y_pred[:, 31] - y_true[:, 31])
  return tf.reduce_mean(diff)

def mean_abs_error_32(y_true, y_pred):
  diff = tf.abs(y_pred[:, 32] - y_true[:, 32])
  return tf.reduce_mean(diff)

def mean_abs_error_33(y_true, y_pred):
  diff = tf.abs(y_pred[:, 33] - y_true[:, 33])
  return tf.reduce_mean(diff)

def mean_abs_error_34(y_true, y_pred):
  diff = tf.abs(y_pred[:, 34] - y_true[:, 34])
  return tf.reduce_mean(diff)

def mean_abs_error_35(y_true, y_pred):
  diff = tf.abs(y_pred[:, 35] - y_true[:, 35])
  return tf.reduce_mean(diff)

def mean_abs_error_36(y_true, y_pred):
  diff = tf.abs(y_pred[:, 36] - y_true[:, 36])
  return tf.reduce_mean(diff)

def mean_abs_error_37(y_true, y_pred):
  diff = tf.abs(y_pred[:, 37] - y_true[:, 37])
  return tf.reduce_mean(diff)

def mean_abs_error_38(y_true, y_pred):
  diff = tf.abs(y_pred[:, 38] - y_true[:, 38])
  return tf.reduce_mean(diff)

def mean_abs_error_39(y_true, y_pred):
  diff = tf.abs(y_pred[:, 39] - y_true[:, 39])
  return tf.reduce_mean(diff)

In [ ]:
"""
Currently the best performance is when having a 4-unit layer in between with LeakyRelu activations, SGD optimizer with LR of 0.001.
In one of the random restarts, it reached ~ 0.93.
It was better with augmentation of GaussianNoise(0.3) and above learning rate schedule [0.01, 5000, 0.96].
"""

int_sequences_input = keras.Input(shape=(1,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = embedded_sequences
# x = layers.GlobalMaxPooling1D()(x)
x = layers.Lambda(lambda xx: tf.squeeze(xx, axis=-2))(x)
x = layers.Dense(300, activation="relu")(x)
x = layers.Dropout(0.05)(x)
x = layers.Dense(200, activation="relu")(x)
x = layers.Dropout(0.05)(x)
x = layers.Dense(100, activation="relu")(x)
x = layers.Dropout(0.05)(x)
x = layers.Dense(80, activation="relu")(x)
x = layers.Dropout(0.05)(x)
x = layers.Dense(60, activation="relu")(x)
x = layers.Dropout(0.05)(x)
# x = layers.Dense(40, activation="relu")(x)
# x = layers.Dropout(0.05)(x)
preds = layers.Dense(40, activation="sigmoid")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

model.compile(
    loss=tf.keras.losses.MeanAbsoluteError(), optimizer=keras.optimizers.SGD(learning_rate=lr_schedule), 
    metrics=[
            #  mean_abs_error_0,
            #  mean_abs_error_1,
            #  mean_abs_error_2,
            #  mean_abs_error_3,
            #  mean_abs_error_4,
            #  mean_abs_error_5,
            #  mean_abs_error_6,
            #  mean_abs_error_7,
            #  mean_abs_error_8,
            #  mean_abs_error_9,
            #  mean_abs_error_10,
            #  mean_abs_error_11,
            #  mean_abs_error_12,
            #  mean_abs_error_13,
            #  mean_abs_error_14,
            #  mean_abs_error_15,
            #  mean_abs_error_16,
            #  mean_abs_error_17,
            #  mean_abs_error_18,
            #  mean_abs_error_19,
            #  mean_abs_error_20,
            #  mean_abs_error_21,
            #  mean_abs_error_22,
            #  mean_abs_error_23,
            #  mean_abs_error_24,
            #  mean_abs_error_25,
            #  mean_abs_error_26,
            #  mean_abs_error_27,
            #  mean_abs_error_28,
            #  mean_abs_error_29,
            #  mean_abs_error_30,   
            #  mean_abs_error_31,
            #  mean_abs_error_32,
            #  mean_abs_error_33,
            #  mean_abs_error_34,
            #  mean_abs_error_35,
            #  mean_abs_error_36,
            #  mean_abs_error_37,
            #  mean_abs_error_38,
            #  mean_abs_error_39,                                               
            #  #keras.metrics.MeanAbsoluteError(), 
            #  tf.keras.metrics.MeanSquaredLogarithmicError(name="mean_squared_logarithmic_error", dtype=None), 
#             keras.metrics.AUC(multi_label=True),
tf.keras.losses.MeanAbsolutePercentageError(
    reduction="auto", name="mean_absolute_percentage_error")
]
)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, restore_best_weights=True)
history = model.fit(x_train, y_train, epochs=20000, validation_data=(x_test, y_test), verbose=2, callbacks=callback)

In [ ]:
model.load_weights("/content/drive/MyDrive/audio-and-word-embeddings/trained models on 19-11-21/glove-6B-1.h5")

In [ ]:
model.load_weights("/content/drive/MyDrive/audio-and-word-embeddings/trained models on 19-11-21/random-1.h5")

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
#Spacey fold-1
model.evaluate(x_test, y_test)

In [ ]:
#Spacey fold-2
model.evaluate(x_test, y_test)

In [ ]:
#Spacey fold-3
model.evaluate(x_test, y_test)

In [ ]:
model.save_weights("spacey-random-2.h5")

In [ ]:
# Random model evaluation with trainable=True
model.evaluate(x_test, y_test)

In [ ]:
# Random model evaluation with trainable=True 
# fold-2
model.evaluate(x_test, y_test)

In [ ]:
# Random model evaluation with trainable=True 
# fold-3
model.evaluate(x_test, y_test)

In [ ]:
# Evaluate Glove embeddings
model.evaluate(x_test, y_test)

In [ ]:
# Evaluate Glove 840B embeddings
model.evaluate(x_test, y_test)

In [ ]:
# Evaluate Glove 840B embeddings
# Fold-2
model.evaluate(x_test, y_test)

In [ ]:
# Evaluate dict2vec embeddings
model.evaluate(x_test, y_test)

In [ ]:
# Random embedding evaluation with trainable=False
model.evaluate(x_val, y_val)

In [ ]:
# Random initialisation with sigmoid outputs
model.evaluate(x_val, y_val)

In [ ]:
model.evaluate(x_val, y_val)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
# model.evaluate(x_val, y_val)

In [ ]:
# model.evaluate(x_val, y_val)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
model.evaluate(x_train, y_train)

In [ ]:
len(history.history['loss'])

In [ ]:
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.ylabel('Error')
plt.xlabel('No. epoch')
plt.legend()
plt.show()

In [ ]:
#Spacey fold-2
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.ylabel('Error')
plt.xlabel('No. epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.ylabel('Error')
plt.xlabel('No. epoch')
plt.legend()
plt.show()

In [ ]:
# Random fold-2
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.ylabel('Error')
plt.xlabel('No. epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.ylabel('Error')
plt.xlabel('No. epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['mean_absolute_percentage_error'], label='Train')
plt.plot(history.history['val_mean_absolute_percentage_error'], label='Validation')
plt.ylabel('Error')
plt.xlabel('No. epoch')
plt.legend()
plt.show()

In [ ]:
model.save_weights('audio_embedding.h5')

In [ ]:
model.save_weights("/content/drive/MyDrive/audio-and-word-embeddings/E3/model_5.h5")

In [ ]:
model.evaluate(x=x_test, y=y_test)

In [ ]:
model.evaluate(x=x_test, y=y_test)

In [ ]:
model.evaluate(x=x_test, y=y_test)

In [ ]:
model.evaluate(x=x_test, y=y_test)

In [ ]:
model.evaluate(x=x_test, y=y_test)

In [ ]:
# random
model.evaluate(x=x_test, y=y_test)

In [ ]:
# +3 db
model.evaluate(x=x_test, y=y_test)

In [ ]:
# +3 db
model.evaluate(x=x_test, y=y_test)

In [ ]:
model.save_weights("/content/drive/MyDrive/audio-and-word-embeddings/E2/model_3.h5")

In [ ]:
model.save("/content/drive/MyDrive/audio-and-word-embeddings/E2/model_3_backup.h5")

In [ ]:
model.save_weights("/content/drive/MyDrive/audio-and-word-embeddings/E2/model_3_backup_weights.h5")

In [ ]:
model.load_weights("/content/drive/MyDrive/audio-and-word-embeddings/E2/model_3.h5")

# Testing

In [ ]:
test_words = ["metallic"]
test_words_vec = vec2(np.array([[s] for s in test_words])).numpy()
p = model.predict(test_words_vec)
# y_test = (y_test - mmin) / (mmax - mmin)
p = p * (mmax - mmin) + mmin

In [ ]:
p

In [ ]:
p

In [ ]:
p2.shape

In [ ]:
!pip install git+https://github.com/detly/gammatone.git

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1FV5KDNVsBpb1fRp2kmyWxDhL1EfvvqAP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1FV5KDNVsBpb1fRp2kmyWxDhL1EfvvqAP" -O signals.zip && rm -rf /tmp/cookies.txt

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1FV5KDNVsBpb1fRp2kmyWxDhL1EfvvqAP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1FV5KDNVsBpb1fRp2kmyWxDhL1EfvvqAP" -O signals.zip && rm -rf /tmp/cookies.txt
!unzip signals.zip

In [ ]:
from gammatone import filters 
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.display

In [ ]:
# Load and listen to original audio sample

x, fs = sf.read('signals/e_gtr_short.wav')
print(f"Loaded {x.shape[0]} samples at fs={fs}")
IPython.display.Audio(x,rate=fs)

In [ ]:
# A list of terms taken from the SocialFX Dataset
gain_vector = {}
gain_vector['hot'] = np.flip(np.array([0.797209, -1.287298455, -1.532964082, -1.506539385, -1.396973938, -1.271198913, -1.017866669, -0.528729007, 0.165573584, 0.754400175, 1.073992036, 1.282210725, 1.442060062, 1.568685481, 1.653999697, 1.648503806, 1.53967359, 1.580347112, 1.577686577, 1.205675471, 0.676758791, 0.252821123, 0.075167078, -0.033101099, -0.089115047, -0.1365393, -0.143565939, -0.084299287, -0.051027767, -0.038591153, -0.089749952, -0.192244295, -0.345006204, -0.529255497, -0.724989266, -0.864740603, -0.916542181, -0.839557577, -0.823611993, -0.980958406]))
gain_vector['warm'] = np.flip(np.array([0.806225, 1.970152522, 2.063643883, 2.109621503, 2.019063089, 1.833557153, 1.54050257, 1.253791039, 1.124354285, 0.912120468, 0.479917036, -0.176767683, -0.766548101, -0.982288412, -0.876476603, -0.871483715, -0.986591984, -1.095373269, -1.006018325, -0.665307271, -0.386414479, -0.282319476, -0.236480253, -0.182852135, -0.094328345, 0.115664993, 0.429528178, 0.443207819, 0.225592816, -0.061474359, -0.435409903, -0.866104454, -1.093506922, -1.048872922, -0.838433617, -0.650190554, -0.567333094, -0.666182754, -0.760821358, -0.622340305]))
gain_vector['cold'] = np.flip(np.array([0.56672, -2.034095236, -2.054271815, -2.050160073, -1.971859231, -1.880512554, -1.774836615, -1.588498521, -1.323855231, -0.929419044, -0.477990841, 0.001836709, 0.425480213, 0.710446169, 0.736694949, 0.743315275, 0.699904753, 0.573891718, 0.417793993, 0.336596002, 0.362311733, 0.393507005, 0.31382609, 0.206445573, 0.156956594, 0.131116367, 0.096204939, 0.101053811, 0.198440373, 0.395838345, 0.665566916, 0.984723167, 1.146000944, 0.996436067, 0.733647623, 0.466518535, 0.551724202, 0.883208881, 0.863160135, 0.817721258]))
gain_vector['relaxing'] = np.flip(np.array([0.667283, 2.12661933, 1.972813587, 2.076840264, 2.029614099, 1.828261577, 1.529357681, 1.106721532, 0.574096502, 0.036519114, -0.369146913, -0.763306351, -1.214185152, -1.591751218, -1.730069773, -1.572405592, -1.283399138, -1.112799795, -0.831637822, -0.283981666, 0.018265935, 0.070173219, -0.021844884, -0.107969439, -0.090211942, 0.099998408, 0.417992371, 0.497223147, 0.322845287, 0.05663085, -0.207602229, -0.375132475, -0.362059187, -0.237172682, -0.145072176, -0.271109014, -0.57736182, -0.58486884, -0.355700261, -0.28738431]))
gain_vector['soothing'] = np.flip(np.array([0.668623, -0.342018362, 0.026666401, 0.446729242, 0.870928285, 1.273066751, 1.605112245, 1.788250756, 1.713096735, 1.410331198, 0.944353776, 0.307533749, -0.378830743, -1.014190533, -1.536209856, -1.920257823, -2.106073864, -2.078396015, -1.942651727, -1.623318874, -1.065090653, -0.478780769, 0.026683737, 0.352843499, 0.58902601, 0.684407324, 0.492285815, 0.224337514, 0.199443126, 0.277549186, 0.356415253, 0.388200774, 0.289154264, 0.096006927, 0.001571427, 0.070259291, 0.231924942, 0.189760673, -0.05734363, -0.202312582]))
gain_vector['harsh'] = np.flip(np.array([0.444456, -0.763625387, -1.097657151, -1.109271826, -0.828296799, -0.504376203, -0.175019514, 0.112461643, 0.343172372, 0.48092289, 0.578483124, 0.793573687, 1.1619776, 1.550687911, 1.789115385, 1.813865965, 1.528659423, 1.337018759, 1.303155904, 1.032093521, 0.634841813, 0.406146777, 0.415351652, 0.425582285, 0.305761841, 0.050915992, -0.345361682, -0.840566551, -1.239371225, -1.504682027, -1.659385274, -1.677605366, -1.570754566, -1.349434626, -1.050155243, -0.736486167, -0.341679134, -0.02872275, 0.067719463, 0.195935989]))


In [ ]:
"cold" in train_word_set

In [ ]:
# Create filters
# - I have checked the center frequencies and implementation of the filters, and they all match up the same from the paper.
c_freqs = filters.centre_freqs(fs, 40, 20)
fcoefs = filters.make_erb_filters(fs, c_freqs, width=1.0)

In [ ]:
gain_vector['hot'].shape

In [ ]:
c_freqs

In [ ]:
test_word_set

In [ ]:
val_word_set

In [ ]:
train_word_set

In [ ]:
model.load_weights("/content/spacey-2.h5")

In [ ]:
"harsh" in train_word_set

In [ ]:
# Apply filterbank to audio signal and apply a given gain waiting value  to each band.
# Select a term (from the list above) and listen.
filterbank_x = filters.erb_filterbank(x, fcoefs)
out = np.sum(filterbank_x.T * gain_vector['metal'],axis=1)
IPython.display.Audio(out,rate=fs)

In [ ]:
"hot" in train_word_set

In [ ]:
len(val_word_set)

In [ ]:
test_words = ["harsh"]
test_words_vec = vec2(np.array([[s] for s in test_words])).numpy()
p = model.predict(test_words_vec)
# y_test = (y_test - mmin) / (mmax - mmin)
p_harsh = p * (mmax - mmin) + mmin

In [ ]:
mmax - mmin

In [ ]:
p_harsh

In [ ]:
test_words_vec

In [ ]:
# Apply filterbank to audio signal and apply a given gain waiting value  to each band.
# Select a term (from the list above) and listen.
filterbank_x = filters.erb_filterbank(x, fcoefs)
out = np.sum(filterbank_x.T * np.flip(p[0, :]),axis=1)
IPython.display.Audio(out,rate=fs)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
p_sharp.shape

In [ ]:
plt.plot(np.flip(gain_vector['hot']), label="hot")
plt.plot(p_hot[0, :], label="pred_hot")
plt.plot(p_hot_2[0, :], label="pred_hot_random")
plt.legend()

In [ ]:
plt.plot(np.flip(gain_vector['harsh']), label="harsh")
plt.plot(p_harsh[0, :], label="pred_harsh")
plt.plot(p_harsh_2[0, :], label="pred_harsh_random")
plt.legend()

In [ ]:
abs = tf.keras.losses.MeanAbsoluteError(
    reduction="auto", name="mean_absolute_error"
)

In [ ]:
abs(np.flip(gain_vector['harsh']), p_harsh[0, :]).numpy()

In [ ]:
abs(np.flip(gain_vector['harsh']), p_harsh_2[0, :]).numpy()

In [ ]:
plt.plot(p_bright[0, :], label="bright")
plt.plot(p_warm[0, :], label="warm")
plt.legend()

In [ ]:
plt.plot(p_harsh[0, :], label="harsh")
plt.plot(p_warm[0, :], label="warm")
plt.legend()

In [ ]:
plt.plot(p_metal[0, :], label="metal")
plt.plot(p_warm[0, :], label="warm")
plt.legend()

In [ ]:
# Apply filterbank to audio signal and apply a given gain waiting value  to each band.
# Select a term (from the list above) and listen.
filterbank_x = filters.erb_filterbank(x, fcoefs)
out = np.sum(filterbank_x.T * p2[0, :-1],axis=1)
IPython.display.Audio(out,rate=fs)

In [ ]:
"sharp" in train_word_set

# Testing with audio files

In [ ]:
# Load and listen to original audio sample

x, fs = sf.read('signals/e_gtr_short.wav')
print(f"Loaded {x.shape[0]} samples at fs={fs}")
IPython.display.Audio(x,rate=fs)

In [ ]:
test_words = ["harsh"]
test_words_vec = vec2(np.array([[s] for s in test_words])).numpy()
p = model.predict(test_words_vec)
# y_test = (y_test - mmin) / (mmax - mmin)
p = p * (mmax - mmin) + mmin

# Apply filterbank to audio signal and apply a given gain waiting value  to each band.
# Select a term (from the list above) and listen.
filterbank_x = filters.erb_filterbank(x, fcoefs)
out = np.sum(filterbank_x.T * np.flip(p[0, :]),axis=1)
IPython.display.Audio(out,rate=fs)

In [ ]:
test_words = ["cold"]
test_words_vec = vec2(np.array([[s] for s in test_words])).numpy()
p2 = model.predict(test_words_vec)
# y_test = (y_test - mmin) / (mmax - mmin)
p2 = p2 * (mmax - mmin) + mmin

# Apply filterbank to audio signal and apply a given gain waiting value  to each band.
# Select a term (from the list above) and listen.
filterbank_x = filters.erb_filterbank(x, fcoefs)
out = np.sum(filterbank_x.T * np.flip(p2[0, :-1]),axis=1)
IPython.display.Audio(out,rate=fs)

In [ ]:
p == p2

In [ ]:
"loud" in train_word_set